In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier as xgb_s
from sklearn.metrics import f1_score
%matplotlib inline
%pylab inline
from matplotlib import pyplot as plt

from sklearn import cross_validation, datasets, linear_model, metrics
from sklearn.preprocessing import StandardScaler
from sklearn import ensemble, learning_curve, metrics 
import numpy as np
import xgboost as xgb

Populating the interactive namespace from numpy and matplotlib


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
data_train_raw = pd.read_csv('E:\\ya_ATMchallenge\\train_data_prepared.csv', sep=';')
data_test_raw = pd.read_csv('E:\\ya_ATMchallenge\\test_data_prepared.csv', sep=';')
#data_train_raw.head()

Задачи, которые я собираюсь здесь решить:
- визуализировать данные, а именно:
 - графики
- попытаться убрать лишние признаки и оценить те данные, на которые больше всего завязывается модель
- (DONE) масштабирование признаков
- (DONE) оставить в модели категориальные признаки 
- применить все известные мне алгоритмы для классификации
- реализовать поиск по сетке


Задача: оставить в модели категориальные признаки

In [3]:
#смотрим, какие признаки объекты и их меняем на числа
#print('{:30} {}'.format('column', 'nunique'))
#for col in data_train_raw.dtypes[data_train_raw.dtypes == 'object'].index:
#    print('{:30} {}'.format(col, data_train_raw[col].nunique()))
    
data_train=data_train_raw.copy()
data_train['cash_in_out']=data_train.cash_in_out.apply(lambda x : 0 if x=='Cash_Out' else 1)
data_train['display_type']=data_train.display_type.apply(lambda x : 0 if x=='Buttons' else 1)
data_train['scanner_code_reader']=data_train.scanner_code_reader.apply(lambda x : 0 if x=='2D' else 1)

data_test=data_test_raw.copy()
data_test['cash_in_out']=data_test.cash_in_out.apply(lambda x : 0 if x=='Cash_Out' else 1)
data_test['display_type']=data_test.display_type.apply(lambda x : 0 if x=='Buttons' else 1)
data_test['scanner_code_reader']=data_test.scanner_code_reader.apply(lambda x : 0 if x=='2D' else 1)

In [4]:
#искуственное разбиение для проверок с бейзлайном
data_train_trainPart, data_train_testPart = data_train[data_train.period < 20], data_train[data_train.period > 19]

Задача - отмасштабировать признаки

In [5]:
#стандартный scaler
scaler = StandardScaler()
data_train_target = data_train.target
data_train = scaler.fit_transform(data_train.drop(['target', 'period'], axis=1))
data_test = scaler.transform(data_test.drop(['period'], axis=1))

data_train_trainPart_target = data_train_trainPart.target
data_train_testPart_target = data_train_testPart.target
data_train_trainPart = scaler.fit_transform(data_train_trainPart.drop(['target', 'period'], axis=1))
data_train_testPart = scaler.transform(data_train_testPart.drop(['target','period'], axis=1))

#### RidgeClassifier + LogisticRegression
Стандартные настройки

In [ ]:
#создание объекта - классификатора
ridge_classifier = linear_model.RidgeClassifier(random_state = 1)
ridge_scoring = cross_validation.cross_val_score(ridge_classifier, data_train, data_train_target, scoring = 'f1', cv = 10, n_jobs = -1)
print (ridge_scoring)
print ('Ridge mean: {}, max:{}, min:{}, std:{}'.format(ridge_scoring.mean(), ridge_scoring.max(), 
                                                     ridge_scoring.min(), ridge_scoring.std()))

In [ ]:
log_regressor = linear_model.LogisticRegression(random_state = 1, n_jobs=-1)
lr_scoring = cross_validation.cross_val_score(log_regressor, data_train, data_train_target, scoring = 'f1', cv = 10, n_jobs = -1)
print (lr_scoring)
print ('Log mean: {}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(), lr_scoring.max(), 
                                                   lr_scoring.min(), lr_scoring.std()))

Теперь попробуем метрику f1 + StratifiedShuffleSplit

In [ ]:
scorer = metrics.make_scorer(metrics.f1_score)
cv_strategy = cross_validation.StratifiedShuffleSplit(data_train_target, n_iter = 20 , test_size = 0.3, random_state = 1)
ridge_scoring = cross_validation.cross_val_score(ridge_classifier, data_train, data_train_target, scoring = scorer, cv = cv_strategy, n_jobs = -1)
lr_scoring = cross_validation.cross_val_score(log_regressor, data_train, data_train_target, scoring = scorer, cv = cv_strategy, n_jobs = -1)

In [ ]:
print (ridge_scoring)
print ('Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring.mean(), ridge_scoring.max(), 
                                                     ridge_scoring.min(), ridge_scoring.std()))
print (lr_scoring)
print ('Log mean:{}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(), lr_scoring.max(), 
                                                   lr_scoring.min(), lr_scoring.std()))

Качество выросло совсем чуть, второй знак после запятой. Попробуем регуляризацию l1 в lr

In [ ]:
log_regressor = linear_model.LogisticRegression(random_state = 1, n_jobs=-1, penalty='l1')
lr_scoring = cross_validation.cross_val_score(log_regressor, data_train, data_train_target, scoring = scorer, cv = cv_strategy, n_jobs = -1)
print (lr_scoring)
print ('С регуляризацией l1: Log mean:{}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(), lr_scoring.max(), 
                                                   lr_scoring.min(), lr_scoring.std()))

Чуть подросло качество еще, +0,03. Пробуем SGD + смотрим отбор параметров.
#### SGDClassifier

In [ ]:
SGDClassifier = linear_model.SGDClassifier(penalty = 'l2', n_jobs = -1)
SGD_scoring = cross_validation.cross_val_score(SGDClassifier, data_train, data_train_target, scoring = scorer, cv = cv_strategy, n_jobs = -1)
print ('SGD с регуляризацией l1: Log mean:{}, max:{}, min:{}, std:{}'.format(SGD_scoring.mean(), SGD_scoring.max(), 
                                                   SGD_scoring.min(), SGD_scoring.std()))

С 2мя регуляризациями качество sgd хуже предыдущих.

### Проведем первичное знакомство с обзором признаков через sgd с регуляризацией l1

In [7]:
SGDClassifier = linear_model.SGDClassifier(penalty = 'l1', n_jobs = -1, random_state = 1)
SGDClassifier.fit(data_train_trainPart, data_train_trainPart_target)
metrics.f1_score(data_train_testPart_target, SGDClassifier.predict(data_train_testPart))


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.6088066368857691

Коэффициенты перед признаками:

In [8]:
coef_weights = np.fromiter(SGDClassifier.coef_[0], dtype=float)
features_name = np.array(data_train_raw.columns.drop(['period','target']))
sorted(zip(features_name, coef_weights), key=lambda x: x[1])[::-1]

[('non_related_to_transactions_gr5', 156.97429744031575),
 ('avg_mcrw_error_gr5', 156.97429744031552),
 ('non_related_to_status_gr5', 156.97429744031552),
 ('non_related_total_errors_gr5', 156.9742974403154),
 ('related_to_non_related_gr5', 156.97429744031507),
 ('related_to_non_related_gr1', 148.90602908747476),
 ('avg_hasp_error', 146.90600562637),
 ('related_to_non_related', 143.7543632855014),
 ('prr_error_by_disticnt_day_gr3', 136.77116285205037),
 ('non_related_total_errors_prev4', 121.14477570502898),
 ('non_related_to_transactions_prev5', 113.3575140095835),
 ('non_related_total_errors_prev1', 111.22358869789173),
 ('avg_bna_accept_count_gr3', 108.92348941191482),
 ('avg_bna_error_gr2', 104.86656339444335),
 ('days_since_last_cardreader_error_gr4', 101.54643738526455),
 ('days_since_last_bna_error_gr4', 82.12996431644657),
 ('avg_ndc_error_prev3', 80.00000765489914),
 ('avg_cardreader_error_gr4', 75.25303544744678),
 ('related_to_non_related_prev5', 74.7839836724246),
 ('avg_bn